In [122]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [90]:
df = pd.read_csv("data.csv")
df = df[['Make','Model','Year','Engine HP','Engine Cylinders','Transmission Type','Vehicle Style','highway MPG','city mpg','MSRP']]
df.columns = df.columns.str.replace(' ', '_').str.lower()
df = df.fillna(0)
df = df.rename(columns={"msrp": "price"})
df.describe()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
count,11914.000000,11914.000000,11914.000000,11914.000000,11914.000000,1.191400e+04
mean,2010.384338,247.941749,5.614655,26.637485,19.733255,4.059474e+04
std,7.579740,110.507669,1.800554,8.863001,8.987798,6.010910e+04
min,1990.000000,0.000000,0.000000,12.000000,7.000000,2.000000e+03
25%,2007.000000,170.000000,4.000000,22.000000,16.000000,2.100000e+04
50%,2015.000000,225.000000,6.000000,26.000000,18.000000,2.999500e+04
75%,2016.000000,300.000000,6.000000,30.000000,22.000000,4.223125e+04
max,2017.000000,1001.000000,16.000000,354.000000,137.000000,2.065902e+06


# Question 1

In [91]:
df['transmission_type'].value_counts()

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

# Question 2

In [92]:
df.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'price'],
      dtype='object')

In [93]:
df[['year', 'engine_hp', 'engine_cylinders','highway_mpg', 'city_mpg','price']].corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


# Question 3

In [94]:
mean = df['price'].mean()
df['above_average'] = (df['price'] > mean).astype(int)

In [112]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)


In [113]:
from sklearn.metrics import mutual_info_score

def calculate_mi(series):
    return mutual_info_score(series, df_train_full.above_average)

df_mi = df_train_full[['make', 'model', 'transmission_type', 'vehicle_style']].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')

df_mi

,MI
model,0.460994
make,0.238724
vehicle_style,0.083390
transmission_type,0.020884


# Question 4

In [114]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values

del df_train['above_average']
del df_val['above_average']

del df_train['price']
del df_val['price']

In [115]:
from sklearn.feature_extraction import DictVectorizer
train_dict = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)
X_train.shape


(7148, 943)

In [116]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)


LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [117]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [118]:
y_pred = model.predict_proba(X_val)[:, 1]

pred = y_pred > 0.5
(y_val == pred).mean()


0.9458665547629039

# Question 5

In [102]:
subset = ['year','engine_hp','transmission_type','city_mpg']
train_dict_small = df_train[subset].to_dict(orient='records')
dv_small = DictVectorizer(sparse=False)
dv_small.fit(train_dict_small)

X_small_train = dv_small.transform(train_dict_small)

model_small = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model_small.fit(X_small_train, y_train)

val_dict_small = df_val[subset].to_dict(orient='records')
X_small_val = dv_small.transform(val_dict_small)
y_pred_small = model_small.predict_proba(X_small_val)[:, 1]
pred_small = y_pred_small > 0.5
acc = (y_val == pred_small).mean()

In [103]:
feats = ['year','engine_hp','transmission_type','city_mpg']

for feat in feats:
    subset = ['year','engine_hp','transmission_type','city_mpg']
    subset.remove(feat) 
    train_dict_small = df_train[subset].to_dict(orient='records')
    dv_small = DictVectorizer(sparse=False)
    dv_small.fit(train_dict_small)

    X_small_train = dv_small.transform(train_dict_small)

    model_small = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model_small.fit(X_small_train, y_train)

    val_dict_small = df_val[subset].to_dict(orient='records')
    X_small_val = dv_small.transform(val_dict_small)
    y_pred_small = model_small.predict_proba(X_small_val)[:, 1]
    pred_small = y_pred_small > 0.5
    print(feat + ': ' + str(acc - (y_val == pred_small).mean()))

year: -0.00041963911036513313
engine_hp: 0.14057910197230383
transmission_type: 0.002937473772555599
city_mpg: 0.008392782207301663


# Question 6

In [123]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

y_train = np.log(df_train.price.values)
y_val = np.log(df_val.price.values)

del df_train['above_average']
del df_val['above_average']

del df_train['price']
del df_val['price']

In [124]:
from sklearn.feature_extraction import DictVectorizer
train_dict = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)
X_train.shape

(7148, 943)

In [129]:
from sklearn.linear_model import Ridge

model = Ridge(solver='sag', random_state = 42,  max_iter=1000)
model.fit(X_train,y_train)

C:\Users\Wim\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Ridge(max_iter=1000, random_state=42, solver='sag')

In [131]:
from sklearn.metrics import mean_squared_error


val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

y_pred = model.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)


0.48688379302159196

In [132]:
alphas = [0, 0.01,0.1,1,10]

for alpha in alphas:
    model = Ridge(alpha = alpha, solver='sag', random_state = 42,  max_iter=1000)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print(round(rmse,3))

C:\Users\Wim\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.487


C:\Users\Wim\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.487


C:\Users\Wim\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.487


C:\Users\Wim\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.487
0.487


C:\Users\Wim\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
